<a href="https://colab.research.google.com/github/awaiskhan005/DEEP-LEARNING-AND-FORECASTING/blob/main/BTC100_paths_fince_tuned_640.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install properscoring

In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from sklearn.preprocessing import MinMaxScaler
import json
import datetime
import time
from google.colab import drive
from properscoring import crps_ensemble

# Mount Google Drive
drive.mount('/content/drive')

# Fetch Bitcoin data with more history
def fetch_btc_data(interval='5m', period='60d'):
    btc = yf.download(tickers='BTC-USD', interval=interval, period=period, auto_adjust=True)
    return btc

# Prepare data for LSTM with additional features
def prepare_data(df, lookback=60):
    scaler = MinMaxScaler()
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df.fillna(method='bfill', inplace=True)
    scaled_data = scaler.fit_transform(df[['Close', 'SMA_10', 'SMA_20']])
    X, y = [], []
    for i in range(lookback, len(scaled_data) - 1):
        X.append(scaled_data[i - lookback:i])
        y.append(scaled_data[i, 0])
    return np.array(X), np.array(y), scaler

# Build LSTM model with fine-tuned architecture
def build_lstm_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        BatchNormalization(),
        Dropout(0.3),
        LSTM(128, return_sequences=True),
        BatchNormalization(),
        Dropout(0.3),
        LSTM(64, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),
        Dense(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

# Predict next 24 hours with enhanced LSTM
def predict_future(model, data, scaler, steps=288, num_paths=100):
    predictions = []
    current_input = data[-1]
    for path_num in range(num_paths):
        print(f"Generating prediction path {path_num+1}/{num_paths}...")
        temp_input = current_input.copy()
        pred_path = []

        for step in range(steps):
            pred = model.predict(temp_input.reshape(1, *temp_input.shape), verbose=0)[0, 0]
            new_row = np.hstack([pred.reshape(-1, 1), temp_input[-1, 1:].reshape(1, -1)])
            pred_price = scaler.inverse_transform(new_row)[0, 0]
            pred_path.append(pred_price)
            temp_input = np.vstack([temp_input[1:], new_row])

            if step % 50 == 0:
                print(f"Step {step}/{steps} for path {path_num+1}...")

        predictions.append(pred_path)

    print("Prediction complete!")
    return np.array(predictions)

# Save predictions to Google Drive
def save_predictions(predictions, timestamps, filename=None):
    if filename is None:
        filename = f'/content/drive/MyDrive/predictions_{datetime.datetime.utcnow().strftime("%Y%m%d_%H%M%S")}.json'
    with open(filename, 'w') as f:
        json.dump({'timestamps': timestamps, 'predictions': predictions.tolist()}, f)
    print(f"Predictions saved in Google Drive: {filename}")
    return filename

# Fetch real BTC prices after 24 hours and calculate CRPS
def fetch_real_prices_and_calculate_crps(predictions_file):
    btc_real = fetch_btc_data(interval='5m', period='1d')
    real_prices = btc_real['Close'].values[-288:]

    with open(predictions_file, 'r') as f:
        data = json.load(f)
        predictions = np.array(data['predictions'])
        timestamps = data['timestamps']

    crps_score = calculate_crps_for_miner(predictions, real_prices, 300)
    print(f"CRPS Score: {crps_score}")
    plot_real_vs_predicted(predictions, real_prices, timestamps)
    return crps_score

# Plot predicted vs actual prices
def plot_real_vs_predicted(predictions, real_prices, timestamps):
    fig = go.Figure()
    for i, path in enumerate(predictions):
        fig.add_trace(go.Scatter(x=timestamps, y=path, mode='lines', name=f'Path {i+1}'))
    fig.add_trace(go.Scatter(x=timestamps, y=real_prices, mode='lines', name='Actual Price', line=dict(color='red')))
    fig.update_layout(title='Bitcoin Price Prediction vs Actual', xaxis_title='Time', yaxis_title='Price')
    fig.show()

# Main workflow
btc_data = fetch_btc_data()
X, y, scaler = prepare_data(btc_data)
model = build_lstm_model(X.shape[1:])
model.fit(X, y, epochs=20, batch_size=64, verbose=1)
future_predictions = predict_future(model, X, scaler, num_paths=100)
timestamps = [(datetime.datetime.utcnow() + datetime.timedelta(minutes=5 * i)).isoformat() for i in range(288)]
predictions_filename = save_predictions(future_predictions, timestamps)
print("Predictions saved and plotted successfully!")

# Manually run the below after 24 hours
# crps_score = fetch_real_prices_and_calculate_crps(predictions_filename)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[*********************100%***********************]  1 of 1 completed
<ipython-input-3-d159e2d4098b>:28: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


Epoch 1/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 0.4660
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0396
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0132
Epoch 4/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0067
Epoch 5/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0049
Epoch 6/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0035
Epoch 7/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0040
Epoch 8/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0033
Epoch 9/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0032
Epoch 10/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0037
Epoch 11/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 0.0029
Epoch 12/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0031
Epoch 13/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0030
Epoch 14/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.0034
Epoch 15/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
predictions_filename = "/content/drive/MyDrive/predictions_20250304_135126.json"  # Update with actual filename
crps_score = fetch_real_prices_and_calculate_crps(predictions_filename)
